# SudachiとchiTraを使う

このノートブックでは、
- 日本語形態素解析器Sudachi
- 事前学習済み言語モデル+トークナイザchiTra

の二つを実際に利用し、何が行えるのかを理解することがゴールです。

## 準備

### 出力の消去
配布のノートブックでは、各セルの実行結果を参照用に残しています。

作業においては実行場所がわかりにくくなるので、右クリックのメニューから`Clear All Outputs`を実行して消去します。

本来どのようになるのか確認したい、初期状態に戻したいなどの場合は、
ターミナルからコマンド `tar -xvf notebooks.tar.gz` で再度展開を行ってください（ファイルの上書きにご注意ください）。

### chiTra モデルのダウンロード
chiTraのモデルデータは[公式github](https://github.com/WorksApplications/SudachiTra)にて配布されています。
通常はそちらからダウンロードして利用することになります。

今回はこのノートブックとともにs3からダウンロードしているので、ここではパスのみ設定します。

In [1]:
chitra_path = "./chiTra-1.0"

## Sudachi

### Sudachi とは
Sudachi は日本語形態素解析器です。

形態素解析とはテキストを形態素（~= 単語）単位に分割する操作です。

これによって語の区切りを計算機で扱えるようにするほか、辞書データから品詞や正規形といった追加情報も取得できます。

### 基本的な使い方

Sudachi の基本的な使い方は、
- トークナイザを構成する
- 処理したいテキストを渡す

の二段階です。

In [2]:
import sudachipy

# 辞書を読み込む
dictionary = sudachipy.Dictionary()

# トークナイザを生成
tokenizer = dictionary.create()

好きな文章を解析させてみてください。

In [3]:
# 処理対象となる文章を設定
text = "吾輩は猫である。"

# 解析を実行
morphemes = tokenizer.tokenize(text)

# 空白区切りで表示
print(morphemes)

吾輩 は 猫 で ある 。


### 語の情報を取得する

分割を行うのみでなく、辞書から語の情報を取得することもできます。
どのような情報が得られるのか確認してみます。

In [4]:
text = "吾輩は猫である。"
morphemes = tokenizer.tokenize(text)

# 最初の形態素について表示
m = morphemes[0]

print(f"表層形：{m.surface()}")
print(f"辞書形：{m.dictionary_form()}")
print(f"正規形：{m.normalized_form()}")
print(f"読み：{m.reading_form()}")
print(f"品詞：{m.part_of_speech()}")

表層形：吾輩
辞書形：吾輩
正規形：我が輩
読み：ワガハイ
品詞：('代名詞', '*', '*', '*', '*', '*')


これらの情報を使うと、例えば以下のようなことができます。

In [5]:
text = "吾輩は猫である。名前はまだない。"

morphemes = tokenizer.tokenize(text)

# 読みに変換
print("".join(m.reading_form() for m in morphemes))

# 名詞のみを取り出す
print([m.surface() for m in morphemes if "名詞" == m.part_of_speech()[0]])

ワガハイハネコデアル。ナマエハマダナイ。
['猫', '名前']


### 分割単位の変更

Sudachi には複数の分割単位が用意されています。
これによりタスクでの必要に応じて、より細かい/大きな単位での分割を行うことができます。

どのような差があるか見てみましょう。

In [6]:
# 各分割単位のトークナイザを用意
tokA = dictionary.create(mode=sudachipy.SplitMode.A)
tokB = dictionary.create(mode=sudachipy.SplitMode.B)
tokC = dictionary.create(mode=sudachipy.SplitMode.C)

In [7]:
text = "外国人参政権"

# 各分割単位で解析を実行する
print("A単位:", tokA.tokenize(text))
print("B単位:", tokB.tokenize(text))
print("C単位:", tokC.tokenize(text))

A単位: 外国 人 参政 権
B単位: 外国人 参政権
C単位: 外国人参政権


## chiTra

### chiTra とは

chiTra は事前学習済み言語モデルと、それを利用するためのトークナイザを提供します。

#### 事前学習済み言語モデルとは
自然文はそのままでは計算機で扱うのに適していないため、何らかの方法で数値ベクトルに変換するのが一般的です。
例えば形態素解析で分割し、各単語の出現数をカウントするといった方法があります。

近年はこの変換からタスクへの応用までをニューラルモデルで一括して行う手法が注目されています。
モデルをゼロから学習するのは大変なので、"汎用的に良い"変換を大規模なテキストデータから
事前に学習させておいたものをベースとして使うのが一般的で、chiTraモデルもその一つです。

さらに詳しくは、[過去のテックトーク](https://worksapplications.connpass.com/event/244032/)の資料もご参照ください。

### chiTra トークナイザ

chiTra モデルはテキストをそのまま扱うわけではなく、
モデル学習時に登録しているトークンの列を入力として受け取ります。

テキストをこの列に変換する役割を持つのが chiTra トークナイザです。

#### テキストの変換

テキストがどのようなトークン列に変換されるかを見てみましょう

In [8]:
import sudachitra as chitra

# モデルデータからトークナイザを読み込む
tokenizer = chitra.BertSudachipyTokenizer.from_pretrained(chitra_path)

In [9]:
text = "吾輩は猫である。名前はまだない。"

# 文章をトークンの列に分割する
print(tokenizer.tokenize(text))

['我が', '##輩', 'は', '猫', 'で', 'ある', '。', '名前', 'は', '未だ', 'ない', '。']


実際には語の列ではなく語の番号の列（と補助情報）が渡されることになります。
トークナイザを関数として使用すると、モデルに実際に渡される形式での出力を確認できます。

In [10]:
text = "吾輩は猫である。名前はまだない。"

tokenizer(text)

{'input_ids': [2, 14108, 6619, 485, 2851, 477, 10149, 419, 10793, 485, 11207, 10160, 419, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

#### word_form_type について

分割後のトークンの一部が、元のテキストとは違う表記に変換されていることにお気づきでしょうか。

chiTraトークナイザは、辞書情報に基づいて語の正規化を行う機能を持っています。
今使用しているchiTra-1.0は `normalized_and_surface` モードが採用されており、
活用しない語（名詞など）は表記にかかわらず共通の形で出力されるようになっています。

ここでは他のモードではどのような出力になるかを見てみましょう。

なお以下では簡単のため chiTra-1.0 のデータを使用しますが、本来は設定ごとに構築する必要があります。

In [11]:
# surface: 正規化なし
tok1 = chitra.BertSudachipyTokenizer.from_pretrained(chitra_path, word_form_type="surface")

# normalized_and_surface: 活用しない語のみ正規化
tok2 = chitra.BertSudachipyTokenizer.from_pretrained(chitra_path, word_form_type="normalized_and_surface")

# normalized: 全て正規化
tok3 = chitra.BertSudachipyTokenizer.from_pretrained(chitra_path, word_form_type="normalized")

In [12]:
text = "引越してからすだちを届けます"

print(tok1.tokenize(text))
print(tok2.tokenize(text))
print(tok3.tokenize(text))

['引', '##越', 'し', 'て', 'から', 'す', '##だ', '##ち', 'を', '届け', 'ます']
['引っ越し', 'し', 'て', 'から', '酢', '##橘', 'を', '届け', 'ます']
['引っ越し', '為', '##る', 'て', 'から', '酢', '##橘', 'を', '届け', '##る', 'ます']


この正規化により、以下のような利点が期待されます。
- 入力文での語の表記ゆれをトークナイザの段階で統一し、モデルでは同じ語彙として扱える。
- これに伴って統一される分だけ語彙を多く扱うことができる。

逆に以下のようなことも想定されるので、タスクに合わせて選択するとよいでしょう。
- 表記の違いによるニュアンスが失われる（例：人、ひと、ヒト）。
- モードによっては語の活用などの情報が失われる。

### chiTra モデル

次に事前学習済み言語モデルchiTraを使ってみます。

chiTraモデルは HuggingFaceの Transformersフレームワークを採用しています。
今回はこのフレームワークの機能を経由してモデルを利用します。

#### 生の出力

前節でchiTraモデルによって入力テキストをベクトルに変換できるという説明をしました。
まずはどのようなベクトルになるのか、生の出力を見てみましょう。

In [13]:
import transformers

model = transformers.BertModel.from_pretrained(chitra_path)
tok = chitra.BertSudachipyTokenizer.from_pretrained(chitra_path)

Some weights of the model checkpoint at ./chiTra-1.0 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
text = "吾輩は猫である。名前はまだない。"

inputs = tok(text, return_tensors="pt")
print("トークンの数:", inputs["input_ids"].shape[1])

result = model(**inputs)
print("最終層のサイズ:", result.last_hidden_state.shape)
print("最終層の出力:", result.last_hidden_state)

トークンの数: 14
最終層のサイズ: torch.Size([1, 14, 768])
最終層の出力: tensor([[[ 1.1296, -1.3310, -1.2289,  ..., -2.3355, -1.0474, -0.3329],
         [ 0.2558, -0.5519, -0.0475,  ...,  0.9909,  0.8790,  2.5155],
         [-0.9614, -1.9350, -0.5023,  ..., -0.0623, -1.0334,  2.0975],
         ...,
         [ 1.1879, -2.0290, -1.3328,  ..., -0.8317, -1.8916,  1.0683],
         [ 1.1005, -1.6217,  0.0045,  ...,  0.3315, -0.4536,  2.7701],
         [ 1.2624, -2.4691, -1.7452,  ..., -0.7947, -0.1858,  1.8848]]],
       grad_fn=<NativeLayerNormBackward0>)


モデル最終層の出力のサイズから、各トークンが 768次元のベクトルに変換されていることがわかります。
これがさらに別の層に渡され、個別のタスクを学習・実行するもととなります。

とはいえベクトルのままでは人目には理解できないので、次に実際の応用を見てみましょう。

#### マスクされた語を予測する

chiTraモデルはBERTというニューラルモデルのアーキテクチャを採用しています。
これは「二つの文が連続したものかを判定する」「文中のマスクされた語を予測する」の二つのタスクを用いて学習されるモデルです。

したがってこれらのタスクについては追加の学習なしでもタスクを動作させることができます。
ここではこのうち「マスクされた語の予測」を実際に実行します。

In [15]:
model = transformers.BertForMaskedLM.from_pretrained(chitra_path)

fillmask = transformers.FillMaskPipeline(model, tok)

Some weights of the model checkpoint at ./chiTra-1.0 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
# 予測したい部分を「[MASK]」に置き換えたテキスト
text = "吾輩は[MASK]である"

fillmask(text)

[{'score': 0.0613539032638073,
  'token': 17527,
  'token_str': 'ペ ッ ト',
  'sequence': '我が輩 は ペット で 或る'},
 {'score': 0.051257725805044174,
  'token': 2851,
  'token_str': '猫',
  'sequence': '我が輩 は 猫 で 或る'},
 {'score': 0.011168411001563072,
  'token': 27947,
  'token_str': 'プ ロ グ ラ マ ー',
  'sequence': '我が輩 は プログラマー で 或る'},
 {'score': 0.009473491460084915,
  'token': 10732,
  'token_str': '人 間',
  'sequence': '我が輩 は 人間 で 或る'},
 {'score': 0.008770175278186798,
  'token': 26303,
  'token_str': 'サ ラ リ ー マ ン',
  'sequence': '我が輩 は サラリーマン で 或る'}]

#### 別のタスクへの応用

chiTraモデルが採用するBERTアーキテクチャは、各種タスクへの応用力の高さで知られています。
モデルの出力部分を目的のタスクに合わせて組み替えることで、上で見たような言語の理解を様々なタスクに応用できるものとされています。

しかし、入れ替えのみで即座に高い性能が得られるわけではなく、対象とするタスクのデータを用いて追加の調整が必要になります。
これが次のステップで見るファインチューニングというものです。

このノートブックの最後に、chiTraモデルを文章を分類するタスクのための形式で読み込んで、
そのままでは使えないことを確認します。

In [17]:
# 文のカテゴリを予測する形式でモデルを読み込む
model = transformers.BertForSequenceClassification.from_pretrained(chitra_path)
textclsf = transformers.TextClassificationPipeline(model=model, tokenizer=tok)

Some weights of the model checkpoint at ./chiTra-1.0 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized f

In [18]:
# ラベルとスコアが出力されるが、期待するような規則性はない
text = "吾輩は猫である[SEP]名前はまだない"
print(textclsf(text))

text = "引っ越してからすだちを送ります"
print(textclsf(text))

[{'label': 'LABEL_0', 'score': 0.5666131973266602}]
[{'label': 'LABEL_0', 'score': 0.5520848631858826}]


繰り返しになりますが、この時点ではモデルは入力のトークン列をベクトルに変換するまではできるものの、
それをもとにどのように分類を行うか、の基準を持っていません。
その部分を担うことになるモデル最終層はこの時点ではランダムに初期化されており、したがって出力のラベルもランダムなものになります。

次のノートブックでは、この最終層を学習するファインチューニングを行っていきます。